In [ ]:
import pandas as pd
import os
import glob

os.chdir('C:/Users/natas/Downloads/safety/safety/features/')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv_train = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv_train.to_csv( "df.csv", index=False, encoding='utf-8-sig')

In [19]:
import pandas as pd
import os
import glob

In [20]:
df_label = pd.read_csv("C:/Users/natas/Downloads/safety/safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")

In [21]:
df= pd.read_csv('C:/Users/natas/Downloads/safety/safety/features/df.csv')

In [22]:
df = pd.merge(df, df_label, on='bookingID')


In [23]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label
0,1202590843006,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362.0,0.000000,1
1,1202590843006,3.033,77.0,1.137217,8.975278,3.981050,0.021118,-0.022515,0.024522,541.0,0.000000,1
2,1202590843006,6.173,283.0,1.522818,7.947064,5.075056,0.137567,-0.297055,-0.017418,154.0,2.863286,1
3,1202590843006,3.014,353.0,1.134775,8.932489,3.929421,-0.007959,-0.020543,-0.022986,1384.0,0.000000,1
4,1202590843006,5.209,340.0,0.858197,9.078037,2.944571,0.061924,0.015219,0.025517,1142.0,9.750134,1


In [24]:
df.shape

(28867662, 12)

In [25]:
import numpy as np
cols = ['bookingID','Accuracy','Bearing', 'acceleration_x', 'acceleration_y', 'acceleration_z',
'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed']
df[cols] = df[cols].astype(np.int64)



In [26]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label
0,1202590843006,3,353,1,8,3,0,0,0,1362,0,1
1,1202590843006,3,77,1,8,3,0,0,0,541,0,1
2,1202590843006,6,283,1,7,5,0,0,0,154,2,1
3,1202590843006,3,353,1,8,3,0,0,0,1384,0,1
4,1202590843006,5,340,0,9,2,0,0,0,1142,9,1


In [27]:
#Logic- Dangerous driving involves looking at acceleration and bearing data. If there is a high acceleration and high change in 
#bearing then you have a case of sharp turns at high speeds--> Dangerous

 

df['accelerationx2']= ((df['acceleration_x']**2) + (df['acceleration_y']**2) + (df['acceleration_z']**2))

df['resultant_acc'] = df['accelerationx2']**0.5 

df['MA_acc'] = df.rolling(window=2)['resultant_acc'].mean()

df['MA_acc_max']= df.groupby('bookingID')['MA_acc'].transform('max')
df['MA_acc_mean']= df.groupby('bookingID')['MA_acc'].transform('mean')
df['MA_acc_skew']= abs(df.groupby('bookingID')['MA_acc'].transform('skew'))
df['MA_acc_var']= df.groupby('bookingID')['MA_acc'].transform('var')


#Check if speed > speed limit. Max speed limit in Singapore is 80-90 km/h ` 25m/s
df['Speedmax'] = df.groupby('bookingID')['Speed'].transform('max')
df['Speedabovelimit'] = df['Speedmax'].apply(lambda x: 1 if x >25 else 0)

#change in bearing

df['bearing_chg'] = df.rolling(window=2)['Bearing'].mean() #assumption all bearings are vs true north measure clockwise"
df['bearing_chg_max']= df.groupby('bookingID')['bearing_chg'].transform('max')
df['bearing_chg_var']= df.groupby('bookingID')['bearing_chg'].transform('var')


#combination of acceleration and bearing - rudimenatary feature creation. looking for high acceleration and high bearing
df['bearing_acc_comb1'] = df['MA_acc_max'] +  df['bearing_chg_max']
df['bearing_acc_comb2'] = df['MA_acc_var'] +  df['bearing_chg_var']

In [28]:
df.columns

Index(['bookingID', 'Accuracy', 'Bearing', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed',
       'label', 'accelerationx2', 'resultant_acc', 'MA_acc', 'MA_acc_max',
       'MA_acc_mean', 'MA_acc_skew', 'MA_acc_var', 'Speedmax',
       'Speedabovelimit', 'bearing_chg', 'bearing_chg_max', 'bearing_chg_var',
       'bearing_acc_comb1', 'bearing_acc_comb2'],
      dtype='object')

In [29]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,...,MA_acc_mean,MA_acc_skew,MA_acc_var,Speedmax,Speedabovelimit,bearing_chg,bearing_chg_max,bearing_chg_var,bearing_acc_comb1,bearing_acc_comb2
0,1202590843006,3,353,1,8,3,0,0,0,1362,...,9.295276,0.201328,0.217549,18,0,NaN,358.5,10271.128953,369.774303,10271.346502
1,1202590843006,3,77,1,8,3,0,0,0,541,...,9.295276,0.201328,0.217549,18,0,215.0,358.5,10271.128953,369.774303,10271.346502
2,1202590843006,6,283,1,7,5,0,0,0,154,...,9.295276,0.201328,0.217549,18,0,180.0,358.5,10271.128953,369.774303,10271.346502
3,1202590843006,3,353,1,8,3,0,0,0,1384,...,9.295276,0.201328,0.217549,18,0,318.0,358.5,10271.128953,369.774303,10271.346502
4,1202590843006,5,340,0,9,2,0,0,0,1142,...,9.295276,0.201328,0.217549,18,0,346.5,358.5,10271.128953,369.774303,10271.346502


In [30]:
df = df.drop_duplicates(subset='bookingID', keep='last')

In [31]:
df.head()

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,...,MA_acc_mean,MA_acc_skew,MA_acc_var,Speedmax,Speedabovelimit,bearing_chg,bearing_chg_max,bearing_chg_var,bearing_acc_comb1,bearing_acc_comb2
2040,1202590843006,4,72,1,9,3,0,0,0,1220,...,9.295276,0.201328,0.217549,18,0,202.5,358.5,10271.128953,369.774303,10271.346502
4895,274877907034,3,74,0,9,3,0,0,0,1233,...,9.215379,0.302773,0.139033,28,1,71.5,330.0,2697.204936,340.805594,2697.343969
7917,884763263056,3,184,0,9,2,0,0,0,1179,...,9.258795,0.221676,0.176319,26,1,135.5,312.5,3707.506068,324.399755,3707.682387
9773,1073741824054,3,205,2,9,-1,0,0,0,470,...,9.261271,1.406496,0.282635,19,0,150.0,308.0,2498.619694,321.161277,2498.902329
11391,1056561954943,3,24,0,9,0,0,0,0,840,...,9.321080,0.210218,0.473171,24,0,20.0,358.5,6662.252559,370.464194,6662.725730


In [32]:
df.isnull().values.any() #there shouldnt be any null values

False

In [33]:
from sklearn.preprocessing import label_binarize

df['label']= label_binarize(df['label'], classes=[0,1])

In [34]:
df.shape

(20000, 26)

In [35]:
df = df.drop(['bookingID','Accuracy', 'Bearing', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed','Speedmax','accelerationx2','MA_acc','resultant_acc'], axis=1)


In [37]:
df = df.reindex(np.random.permutation(df.index))

In [38]:
df.columns

Index(['label', 'MA_acc_max', 'MA_acc_mean', 'MA_acc_skew', 'MA_acc_var',
       'Speedabovelimit', 'bearing_chg', 'bearing_chg_max', 'bearing_chg_var',
       'bearing_acc_comb1', 'bearing_acc_comb2'],
      dtype='object')

In [39]:
df.head()

,label,MA_acc_max,MA_acc_mean,MA_acc_skew,MA_acc_var,Speedabovelimit,bearing_chg,bearing_chg_max,bearing_chg_var,bearing_acc_comb1,bearing_acc_comb2
10578376,0,11.226753,9.047542,0.383706,0.378044,0,201.0,309.5,2132.966905,320.726753,2133.344948
27429343,0,11.758502,9.317530,0.332302,0.337709,0,163.5,354.0,4500.083705,365.758502,4500.421414
8571612,0,10.319173,9.283645,0.834295,0.072093,0,183.5,356.5,9757.640704,366.819173,9757.712797
6292236,0,12.059604,9.259639,0.050126,0.249534,0,248.0,339.0,2680.826685,351.059604,2681.076219
24493440,0,10.832639,8.882139,0.001759,0.275757,1,278.0,319.5,5407.064261,330.332639,5407.340018


In [40]:
import seaborn as sns
import numpy as np

corr = df.corr()
sns.heatmap(corr)

In [41]:
from sklearn.model_selection import train_test_split
import numpy as np 
labels = df.label
features = df.drop('label', axis=1)


In [42]:
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed)

In [43]:
print('X_train Shape:', X_train.shape)
print('y_train Shape:', y_train.shape)
print('X_test Shape:', X_test.shape)
print('y_test Labels Shape:', y_test.shape)

X_train Shape: (16000, 10)
y_train Shape: (16000,)
X_test Shape: (4000, 10)
y_test Labels Shape: (4000,)


In [54]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)

In [55]:
np.bincount(y_train)

array([12020, 12020], dtype=int64)

In [56]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000)
# Train the model on training data
rf.fit(X_train, y_train);

In [57]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
y_pred_class = rf.predict(X_test)
y_pred_prob = rf.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

0.5398909089344742

In [47]:
from pprint import pprint
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [48]:
#Hyperparameter tuning for random forest
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [47]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, limited due to computational issues
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 141.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 223.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [48]:
rf_random.best_params_ #output: {'n_estimators': 1400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 40, 'bootstrap': False}

{'n_estimators': 1400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 40, 'bootstrap': False}

In [52]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rfhyper = RandomForestClassifier(n_estimators = 1400, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', max_depth = 40, bootstrap = False)
# Train the model on training data
rfhyper.fit(X_train, y_train);

In [50]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
y_pred_class = rfhyper.predict(X_test)
y_pred_prob = rfhyper.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

0.5264368626275241

In [58]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {'C': [ 0.00001, 0.0001, 0.001, 0.01,0.1, 1, 10, 100]}
LRmodel = RandomizedSearchCV(LogisticRegression(), param_grid, cv=3, scoring='roc_auc')

LRmodel.fit(X_train, y_train)

from sklearn import metrics
from sklearn.metrics import roc_curve, auc
y_pred_class = LRmodel.predict(X_test)
y_pred_prob = LRmodel.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

C:\Users\natas\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solv

0.5552642908600908

In [31]:
from pprint import pprint
print('Parameters currently in use:\n')
pprint(model.get_params())

Parameters currently in use:

{'cv': 3,
 'error_score': 'raise-deprecating',
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'warn',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'warn',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'fit_params': None,
 'iid': 'warn',
 'n_jobs': None,
 'param_grid': [{'classifier': [LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class

In [62]:
import xgboost as xgb

In [ ]:
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed)

In [ ]:
xg_class = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
#XGBoost base model
from numpy import loadtxt
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
y_pred_class = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

In [32]:
#Hyperparameters for XGBoost
random_grid ={
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [37]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='reg:logistic',
                    silent=True, nthread=1)

In [40]:
#Hyperparameter tuning for XGBoost
from sklearn.model_selection import RandomizedSearchCV
from numpy import loadtxt
from xgboost import XGBClassifier

random_search = RandomizedSearchCV(xgb, param_distributions=random_grid, n_iter=5, scoring='roc_auc', n_jobs=4, cv=3, verbose=3, random_state=1001 )
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:   44.6s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=5, n_jobs=4,
          param_distributions={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=3)

In [42]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)


 All results:
{'mean_fit_time': array([14.58235971,  7.70366001, 12.43311063,  9.84930277,  6.73854454]), 'std_fit_time': array([0.01738126, 0.42744374, 0.09868049, 0.49761489, 1.20684042]), 'mean_score_time': array([0.39162103, 0.17134237, 0.29159927, 0.22390485, 0.18224804]), 'std_score_time': array([0.02703627, 0.00069458, 0.0073639 , 0.00736373, 0.02655244]), 'param_subsample': masked_array(data=[0.6, 0.6, 0.6, 0.6, 0.6],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[1, 1, 5, 10, 10],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[5, 3, 5, 4, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[1.5, 2, 0.5, 2, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
   

C:\Users\natas\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\natas\Anaconda3\lib\site-packages\sklearn\util

In [92]:
modeltuned = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=2, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6)
modeltuned.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=2, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6)

In [93]:
y_pred_class = modeltuned.predict(X_test)
y_pred_prob = modeltuned.predict_proba(X_test)[:, 1]
y_pred_prob
metrics.roc_auc_score(y_test, y_pred_class)

0.5269597326714361